### **Bài 13: Bài tập về nhà: Ứng dụng mô hình sequence2sequence cho bài toán sinh văn bản (text generation)**
Tổng quan: Ở bài tập này chúng ta sẽ ôn lại cách xây dựng và sử dụng mô hình seq2seq cho bài toán sinh văn bản.

**1. Chuẩn bị dữ liệu và tiền xử lý**

Trong bài tập này chúng ta sẽ xử lý dữ liệu của bài toán tóm tắt văn bản để thực nghiệm cho bài toán sinh văn bản. Trong bài toán tóm tắt văn bản, input của chương trình sẽ là 1 văn bản dài và output sẽ là 1 văn bản ngắn hơn và chứa những thông tin quan trọng của văn bản đầu vào. Ngược lại với bài toán trên, trong bài toán sinh văn bản, chúng ta muốn input đầu vào là 1 vài keyword hoặc 1 đoạn văn ngắn và output ra 1 đoạn văn dài. Vì thế, ta hoàn toàn có thể sử dụng dữ liệu trong bài toán tóm tắt văn bản để huấn luyện cho bài toán sinh văn bản, với input là câu đã được tóm tắt và output là đoạn văn gốc.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/
!mkdir bai13_home
%cd /content/drive/MyDrive/bai13_home

/content/drive/MyDrive
/content/drive/MyDrive/bai13_home


In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

Bài tập 1: Bạn hãy download 1 bộ dữ liệu tóm tắt văn bản và tiền xử lý chúng. Có thể dùng 1 trong các bộ dữ liệu dưới đây
*   Bộ gigaword : https://drive.google.com/open?id=0B6N7tANPyVeBNmlSX19Ld2xDU1E
*   Bộ CNN/DM : https://s3.amazonaws.com/opennmt-models/Summary/cnndm.tar.gz

Yêu cầu : Sau khi tiền xử lý, chúng ta sẽ có 4 file data gồm :
*   train.input.txt : Chứa các câu tóm tắt dùng để huấn luyện mô hình, thường chiếm 80% kích thước tổng dữ liệu
*   train.output.txt : Chứa các đoạn văn bản gốc ứng với các tóm tắt.
*   valid.input.txt : Chứa các câu tóm tắt dùng để đánh giá mô hình, thường chiếm 10% kích thưởng tổng dữ liệu.
*   valid.output.txt : Chứa các đoạn văn bản gốc ứng vs các tóm tắt.

Lưu ý : Nếu bạn để max_length của dữ liệu quá lớn, thì mô hình của bạn sẽ rất to và có thể gây tràn RAM.






In [ ]:
!wget https://s3.amazonaws.com/opennmt-models/Summary/cnndm.tar.gz
!tar -xvzf cnndm.tar.gz

--2022-06-22 15:35:53--  https://s3.amazonaws.com/opennmt-models/Summary/cnndm.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.226.139
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.226.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 500375629 (477M) [application/x-gzip]
Saving to: ‘cnndm.tar.gz’

cnndm.tar.gz        100%[===================>] 477.20M  51.7MB/s    in 8.5s    

2022-06-22 15:36:02 (56.3 MB/s) - ‘cnndm.tar.gz’ saved [500375629/500375629]

test.txt.src
test.txt.tgt.tagged
train.txt.src
train.txt.tgt.tagged
val.txt.src
val.txt.tgt.tagged


### Câu hỏi 1: Viết hàm tách từ và đọc dữ liệu 

In [ ]:
train_src_file = "train.txt.tgt.tagged"
train_tgt_file = "train.txt.src"
valid_src_file = "val.txt.tgt.tagged"
valid_tgt_file = "val.txt.src"

max_length_target = 50
max_length_source = 10

# Thêm start và end token vào mỗi câu
def preprocess_sentence(sentence):
  sentence = "<start> "+sentence+" <end>"
  return sentence

# Hàm load dữ liệu
def load_data(source_file, target_file, number_of_examples):
  f = open(source_file, "r")
  source_sents = f.readlines()
  f0 = open(target_file, "r")
  target_sents = f0.readlines()
  assert len(source_sents) == len(target_sents)

  source_data, target_data = [], []
  for source_sentence, target_sentence in zip(source_sents[:number_of_examples],
                                              target_sents[:number_of_examples]):
    if len(source_sentence.strip().split()) > max_length_source:
      source_sentence = " ".join(source_sentence.strip().split()[:max_length_source])
    if len(target_sentence.strip().split()) > max_length_target:
      target_sentence = " ".join(target_sentence.strip().split()[:max_length_target])
    
    source_data.append(preprocess_sentence(source_sentence))
    target_data.append(preprocess_sentence(target_sentence))
  
  return source_data, target_data

# Viết hàm tách từ với chuyển chữ in hoa thành in thường
# Hàm tách token
def tokenizer(sentences):
  tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='', oov_token = "unk", lower = True)
  tokenizer.fit_on_texts(sentences)

  sent_tensors = tokenizer.texts_to_sequences(sentences)
  sent_tensors = tf.keras.preprocessing.sequence.pad_sequences(sent_tensors,
                                                         padding='post')
  
  return sent_tensors, tokenizer

# Tổng hợp các hàm trên thành một hàm đọc và xử lý dữ liệu
def create_data(source_path, target_path, number_of_examples):
  source_data, target_data = load_data(source_path, target_path, number_of_examples)
  source_tensors, source_tokenizer = tokenizer(source_data)
  target_tensors, target_tokenizer = tokenizer(target_data)
  return source_tensors, target_tensors, source_tokenizer, target_tokenizer

number_of_examples = 10000
train_src_tensors, train_tgt_tensors, train_src_tokenizer, train_tgt_tokenizer = create_data(train_src_file, train_tgt_file, number_of_examples)
valid_src_tensors, valid_tgt_tensors, _, _ = create_data(valid_src_file, valid_tgt_file, -1)

# max_length_source, max_length_target = train_src_tensors.shape[1], train_tgt_tensors.shape[1]

print(len(train_src_tensors), len(valid_src_tensors))

10000 13367


### Câu hỏi 2: Tạo batch dữ liệu
Sử dụng tf.data.Dataset để tạo dữ liệu huấn luyện**
Hãy xem lại bài tập seq2seq cho bài toán dịch máy để thực hiện cách build data theo batch.

In [ ]:
BUFFER_SIZE = len(train_src_tensors)
BATCH_SIZE = 64

steps_per_epoch = len(train_src_tensors)//BATCH_SIZE
vocab_src_size = len(train_src_tokenizer.word_index)+1
vocab_tgt_size = len(train_tgt_tokenizer.word_index)+1

train_dataset = tf.data.Dataset.from_tensor_slices((train_src_tensors, train_tgt_tensors)).shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)

# Print shape of the first batch
example_input_batch, example_target_batch = next(iter(train_dataset))
example_input_batch.shape, example_target_batch.shape


(TensorShape([64, 12]), TensorShape([64, 52]))

## Câu hỏi 3. Mô hình Seq2Seq với Attention**

Bài tập 3: Hãy viết lại các thành phần Encoder, Attention, Decoder theo cách hiểu của bạn(Sử dụng LuongAttention).
## Câu hỏi 3.1 Xây dựng encoder

In [ ]:
#Viết hàm Encoder với mạng RNN
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, hidden_state_size, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.hidden_state_size = hidden_state_size
    # The embedding layer converts tokens to vectors
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    # The GRU RNN layer processes those vectors sequentially.
    self.simpleRNN = tf.keras.layers.SimpleRNN(self.hidden_state_size,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    # YOUR CODE HERE
    x = self.embedding(x)
    output, state = self.simpleRNN(x, initial_state = hidden)
    # Returns the new sequence and its state.
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.hidden_state_size))

### Câu hỏi 3.2 Xây dựng lớp attention

In [ ]:
class LuongAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(LuongAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    values_transposed = tf.transpose(values, perm=[0, 2, 1])
    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = tf.transpose(tf.matmul(query_with_time_axis, values_transposed) , perm=[0, 2, 1])

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

### Câu hỏi 3.3 Xây dựng lớp decoder

In [ ]:
#Viết hàm Decoder với mạng RNN
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, hidden_state_size, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.hidden_state_size = hidden_state_size
    # The embedding layer convets token IDs to vectors
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.simpleRNN = tf.keras.layers.SimpleRNN(self.hidden_state_size,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = LuongAttention(self.hidden_state_size)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the RNN
    output, state = self.simpleRNN(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

## Câu hỏi 3.4 Xây dựng hàm lỗi và tối ưu

Hàm tối ưu SGD và hàm lỗi Cross Entropy được dùng rất phổ biến trong các mô hình học sâu, trong phần này, chúng ra sẽ dùng lại các hàm đó nhé.

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)
  # Mask off the losses on padding.
  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_) # Return the total
optimizer = tf.keras.optimizers.SGD()

## Câu hỏi 4 Huấn luyện

Giờ đã có đủ nguyên liệu và mô hình rồi, ta hãy cùng huấn luyện 1 mô hình có thể sinh văn bản.

In [ ]:
@tf.function
def train_step(source, target, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(source, enc_hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([train_tgt_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

    for t in range(1, target.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(target[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(target[:, t], 1)
  
  batch_loss = (loss / int(target.shape[1]))
  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return batch_loss


In [ ]:
embedding_dim = 512
hidden_state_size = 512

# YOUR CODE HERE
encoder = Encoder(vocab_src_size, embedding_dim, hidden_state_size, BATCH_SIZE)
decoder = Decoder(vocab_tgt_size, embedding_dim, hidden_state_size, BATCH_SIZE)

checkpoint_dir = './model_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

EPOCHS = 1
print("start training ... ")
print(steps_per_epoch)
for epoch in range(EPOCHS):
  start = time.time()
  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
  for (batch, (source, target)) in enumerate(train_dataset.take(steps_per_epoch)):
    batch_loss = train_step(source, target, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 1 epochs
  checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

start training ... 
156
Epoch 1 Batch 0 Loss 10.0539
Epoch 1 Batch 100 Loss 19.3011
Epoch 1 Loss 19.6054
Time taken for 1 epoch 65.18843841552734 sec



## Câu hỏi 5 Sử dụng mô hình vừa được huấn luyện để sinh văn bản

Hãy sử dụng mô hình vừa được huấn luyện để thực hiện sinh văn bản.

In [ ]:
def predict(input_sentence):
  attention_plot = np.zeros((max_length_target, max_length_source))

  input_sentence = preprocess_sentence(input_sentence)
  inputs = []
  for i in input_sentence.lower().split(' '):
    if i not in train_src_tokenizer.word_index:
      inputs.append(train_src_tokenizer.word_index['unk'])
    else:
      inputs.append(train_src_tokenizer.word_index[i])

  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_source,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, hidden_state_size))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([train_tgt_tokenizer.word_index['<start>']], 0)

  for t in range(max_length_target):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += train_tgt_tokenizer.index_word[predicted_id] + ' '
    # Stop predicting when the model predicts the end token.
    if train_tgt_tokenizer.index_word[predicted_id] == '<end>':
      return result, source_sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)
  return result, input_sentence, attention_plot

input_sentence = "hollywood shores up support for ocean 's thirteen"
original_article = "hollywood is planning a new sequel to adventure flick `` ocean 's eleven , '' with star george clooney set to reprise his role as a charismatic thief in `` ocean 's thirteen , '' the entertainment press said wednesday ."
result, input_sentence, attention_plot = predict(input_sentence)
print('Input: %s' % (input_sentence))
print('Output : {}'.format(result))

Input: <start> hollywood shores up support for ocean 's thirteen <end>
Output : -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- -lrb- 
